In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Load the dataset
df = pd.read_csv('Restaurant_Dataset.csv')

In [ ]:
# Display the column names to verify they match expected names
print("Columns in the dataset:", df.columns)


Columns in the dataset: Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')


In [ ]:
# Ensure the expected columns exist
expected_columns = ['Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu']
for col in expected_columns:
    if col not in df.columns:
        print(f"Column '{col}' not found in the dataset.")
        # Add code here to handle missing columns, if necessary

In [ ]:
# Data Preprocessing
df['Has Table booking'] = df['Has Table booking'].map({'Yes': 1, 'No': 0})
df['Has Online delivery'] = df['Has Online delivery'].map({'Yes': 1, 'No': 0})
df['Is delivering now'] = df['Is delivering now'].map({'Yes': 1, 'No': 0})
df['Switch to order menu'] = df['Switch to order menu'].map({'Yes': 1, 'No': 0})


In [ ]:
# One-hot encoding for 'Cuisines' column
cuisine_dummies = df['Cuisines'].str.get_dummies(sep=', ')
df = pd.concat([df, cuisine_dummies], axis=1)


In [ ]:
# Drop columns not needed for the model
columns_to_drop = ['Restaurant ID', 'Restaurant Name', 'Address', 'Locality', 'Locality Verbose', 'Currency', 'Rating color', 'Rating text', 'Cuisines', 'Country Code', 'Latitude', 'Longitude']
df = df.drop([col for col in columns_to_drop if col in df.columns], axis=1)

In [ ]:
# Convert 'City' to categorical and then to numerical
df['City'] = df['City'].astype('category').cat.codes

In [ ]:
# Normalize/Scale numerical features
scaler = StandardScaler()
numerical_features = ['Average Cost for two', 'Price range', 'Votes']
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:
# Splitting the data
X = df.drop('Aggregate rating', axis=1)
y = df['Aggregate rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Training the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
# Saving the model
joblib.dump(model, 'restaurant_rating_model.pkl')

['restaurant_rating_model.pkl']

In [ ]:
# User Input Function
def get_user_input():
    city = input("Enter City: ")
    cuisines = input("Enter Cuisines (comma separated): ").split(', ')
    average_cost_for_two = float(input("Enter Average Cost for Two: "))
    has_table_booking = input("Has Table Booking (Yes/No): ")
    has_online_delivery = input("Has Online Delivery (Yes/No): ")
    is_delivering_now = input("Is Delivering Now (Yes/No): ")
    switch_to_order_menu = input("Switch to Order Menu (Yes/No): ")
    price_range = int(input("Enter Price Range: "))
    votes = int(input("Enter Votes: "))

    # Create a DataFrame for the input
    input_data = {
        'City': [city],
        'Average Cost for two': [average_cost_for_two],
        'Has Table booking': [1 if has_table_booking == 'Yes' else 0],
        'Has Online delivery': [1 if has_online_delivery == 'Yes' else 0],
        'Is delivering now': [1 if is_delivering_now == 'Yes' else 0],
        'Switch to order menu': [1 if switch_to_order_menu == 'Yes' else 0],
        'Price range': [price_range],
        'Votes': [votes]
    }

    # Add cuisine columns
    for cuisine in cuisine_dummies.columns:
        input_data[cuisine] = [1 if cuisine in cuisines else 0]

    # Convert input_data to DataFrame
    input_df = pd.DataFrame(input_data)

    # Convert 'City' to numerical
    input_df['City'] = input_df['City'].astype('category').cat.codes

    # Normalize numerical features
    input_df[numerical_features] = scaler.transform(input_df[numerical_features])

    return input_df


In [ ]:
# Predict Function
def predict_rating(input_df):
    model = joblib.load('restaurant_rating_model.pkl')
    prediction = model.predict(input_df)
    return prediction[0]

In [ ]:
# Main Function
if __name__ == "__main__":
    user_input = get_user_input()
    rating = predict_rating(user_input)
    print(f"The predicted aggregate rating is: {rating}")

Enter City: Dubai
Enter Cuisines (comma separated): Pakistani, Indian, Sweets
Enter Average Cost for Two: 90
Has Table Booking (Yes/No): No
Has Online Delivery (Yes/No): Yes
Is Delivering Now (Yes/No): Yes
Switch to Order Menu (Yes/No): No
Enter Price Range: 3
Enter Votes: 210
The predicted aggregate rating is: 4.277000000000001
